In [1]:
!wget https://paddle-inference-dist.bj.bcebos.com/Paddle-Inference-Demo/resnet50.tgz

--2025-03-09 12:54:26--  https://paddle-inference-dist.bj.bcebos.com/Paddle-Inference-Demo/resnet50.tgz
Resolving paddle-inference-dist.bj.bcebos.com (paddle-inference-dist.bj.bcebos.com)... 36.110.192.178
Connecting to paddle-inference-dist.bj.bcebos.com (paddle-inference-dist.bj.bcebos.com)|36.110.192.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95119015 (91M) [application/x-gzip]
Saving to: 'resnet50.tgz'

     0K .......... .......... .......... .......... ..........  0%  961K 97s
    50K .......... .......... .......... .......... ..........  0% 1.29M 83s
   100K .......... .......... .......... .......... ..........  0% 3.19M 65s
   150K .......... .......... .......... .......... ..........  0% 4.08M 54s
   200K .......... .......... .......... .......... ..........  0% 4.29M 48s
   250K .......... .......... .......... .......... ..........  0% 10.9M 41s
   300K .......... .......... .......... .......... ..........  0% 5.34M 38s
   350K .......

In [2]:
!tar zxf resnet50.tgz

In [1]:
import os
import numpy as np

In [5]:
os.listdir("./resnet50/")

['inference.pdiparams', 'inference.pdiparams.info', 'inference.pdmodel']

In [2]:
from paddle.inference import Config,create_predictor

C:\Users\86173\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [3]:
# 初始化预测器和带锁的预测器
g_PredictorConfig = None
g_Predictor = None
g_PredictorWithLock = None

In [4]:
import paddle.inference as paddle_infer
# 初始化配置
model_path = "./model/inference.pdmodel"
params_path = "./model/inference.pdiparams"
# config = paddle_infer.Config(model_path, params_path)

# try:
#     # 创建预测器
#     predictor = paddle_infer.create_predictor(config)
#     print("Predictor created successfully!")
# except Exception as e:
#     print(f"Failed to create predictor: {e}")

In [5]:
def init_predictor_config():
    config = Config()
    config.set_model(model_path,params_path)
    config.enable_memory_optim()
    config.disable_glog_info()
    config.set_cpu_math_library_num_threads(10)
    config.mkldnn_enabled()
    config.enable_profile()
    return config

In [6]:
def get_predictor_config():
    """
    获取预测器配置单例
    Returns:

    """
    global g_PredictorConfig
    if not g_PredictorConfig:
        g_PredictorConfig = init_predictor_config()
    return g_PredictorConfig

In [7]:
def init_predictor():
    """
    初始化预测器对象
    Returns:预测器对象
    """
    config = get_predictor_config()
    predictor = create_predictor(config)
    return predictor


In [8]:
def get_predictor():
    """
    获取预测器单例
    Returns:单例预测器
    """
    global g_Predictor
    if not g_Predictor:
        g_Predictor = init_predictor()
    return g_Predictor

In [9]:
def run(pre, img):
    """
    执行预测
    Args:
        pre: 预测器
        img: 图片数据

    Returns: 预测结果

    """
    # copy img data to input tensor
    input_names = pre.get_input_names()
    for i, name in enumerate(input_names):
        input_tensor = pre.get_input_handle(name)
        input_tensor.reshape(img[i].shape)
        input_tensor.copy_from_cpu(img[i])
    # do the inference
    pre.run()
    results = []
    # get out data from output tensor
    output_names = pre.get_output_names()
    for i, name in enumerate(output_names):
        output_tensor = pre.get_output_handle(name)
        output_data = output_tensor.copy_to_cpu()
        results.append(output_data)
    return results

In [10]:
def preprocess(img_bytes: bytes) -> np.ndarray:
    """
    这个函数是一个整体的图像预处理流程，包括调用resize_short和crop_image函数对图像进行缩放和裁剪操作，
    然后进行一系列的处理，如颜色通道转换（BGR到RGB）、维度变换（HWC到CHW）、归一化等操作，
    最终返回经过预处理后的图像数据。
    """
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    img = np.frombuffer(img_bytes, np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)
    img = resize_short(img, 224)
    img = crop_image(img, 224, True)
    # bgr-> rgb && hwc->chw
    img = img[:, :, ::-1].astype('float32').transpose((2, 0, 1)) / 255
    img_mean = np.array(mean).reshape((3, 1, 1))
    img_std = np.array(std).reshape((3, 1, 1))
    img -= img_mean
    img /= img_std
    return img[np.newaxis, :]

In [11]:
def result_process(result):
    max_labels = np.argsort(result[0][0])[::-1][:5]
    res = {}
    data = get_label_data()
    for lab in max_labels:
        key = data[str(lab)]['name']
        value = result[0][0][lab]
        res[key] = value * 100
    for key in res:
        res[key] = round(res[key], 2)

    return res

In [12]:
LABEL_DICT = {
    "0": {
        "pinyin": "aiye",
        "name": "艾叶"
    },
    "1": {
        "pinyin": "ajiao",
        "name": "阿胶"
    },
    "2": {
        "pinyin": "baibiandou",
        "name": "白扁豆"
    },
    "3": {
        "pinyin": "baibu",
        "name": "百部"
    },
    "4": {
        "pinyin": "baifan",
        "name": "白矾"
    },
    "5": {
        "pinyin": "baihe",
        "name": "百合"
    },
    "6": {
        "pinyin": "baihuasheshecao",
        "name": "白花蛇舌草"
    },
    "7": {
        "pinyin": "baikou",
        "name": "白蔻"
    },
    "8": {
        "pinyin": "baimaogen",
        "name": "白茅根"
    },
    "9": {
        "pinyin": "baishao",
        "name": "白芍"
    },
    "10": {
        "pinyin": "baitouweng",
        "name": "白头翁"
    },
    "11": {
        "pinyin": "baizhu",
        "name": "白术"
    },
    "12": {
        "pinyin": "baiziren",
        "name": "柏子仁"
    },
    "13": {
        "pinyin": "bajitian",
        "name": "巴戟天"
    },
    "14": {
        "pinyin": "banlangen",
        "name": "板蓝根"
    },
    "15": {
        "pinyin": "banxia",
        "name": "半夏"
    },
    "16": {
        "pinyin": "beishashenkuai",
        "name": "北沙参块"
    },
    "17": {
        "pinyin": "beishashentiao",
        "name": "北沙参条"
    },
    "18": {
        "pinyin": "biejia",
        "name": "鳖甲"
    },
    "19": {
        "pinyin": "cangzhu",
        "name": "苍术"
    },
    "20": {
        "pinyin": "caoguo",
        "name": "草果"
    },
    "21": {
        "pinyin": "caokou",
        "name": "草寇"
    },
    "22": {
        "pinyin": "cebaiye",
        "name": "侧柏叶"
    },
    "23": {
        "pinyin": "chaihu",
        "name": "柴胡"
    },
    "24": {
        "pinyin": "chantui",
        "name": "蝉蜕"
    },
    "25": {
        "pinyin": "chenpi",
        "name": "陈皮"
    },
    "26": {
        "pinyin": "chenxiang",
        "name": "沉香"
    },
    "27": {
        "pinyin": "chishao",
        "name": "赤芍"
    },
    "28": {
        "pinyin": "chishizhi",
        "name": "赤石脂"
    },
    "29": {
        "pinyin": "chongcao",
        "name": "虫草"
    },
    "30": {
        "pinyin": "chuanshanjia",
        "name": "穿山甲"
    },
    "31": {
        "pinyin": "chuanxinlian",
        "name": "穿心莲"
    },
    "32": {
        "pinyin": "cishi",
        "name": "刺史"
    },
    "33": {
        "pinyin": "dafupi",
        "name": "大腹皮"
    },
    "34": {
        "pinyin": "dangshen",
        "name": "党参"
    },
    "35": {
        "pinyin": "danshen",
        "name": "丹参"
    },
    "36": {
        "pinyin": "daqingye",
        "name": "大青叶"
    },
    "37": {
        "pinyin": "daxueteng",
        "name": "大血藤"
    },
    "38": {
        "pinyin": "digupi",
        "name": "地骨皮"
    },
    "39": {
        "pinyin": "dilong",
        "name": "地龙"
    },
    "40": {
        "pinyin": "diyu",
        "name": "地榆"
    },
    "41": {
        "pinyin": "duzhong",
        "name": "杜仲"
    },
    "42": {
        "pinyin": "fangfeng",
        "name": "防风"
    },
    "43": {
        "pinyin": "foshou",
        "name": "佛手"
    },
    "44": {
        "pinyin": "fuling",
        "name": "茯苓"
    },
    "45": {
        "pinyin": "fupenzi",
        "name": "覆盆子"
    },
    "46": {
        "pinyin": "fuzi",
        "name": "附子"
    },
    "47": {
        "pinyin": "gancao",
        "name": "甘草"
    },
    "48": {
        "pinyin": "ganjiang",
        "name": "干姜"
    },
    "49": {
        "pinyin": "gegen",
        "name": "葛根"
    },
    "50": {
        "pinyin": "gouqizi",
        "name": "枸杞子"
    },
    "51": {
        "pinyin": "gouteng",
        "name": "钩藤"
    },
    "52": {
        "pinyin": "guanzhong",
        "name": "贯众"
    },
    "53": {
        "pinyin": "guya",
        "name": "谷芽"
    },
    "54": {
        "pinyin": "hehuanpi",
        "name": "合欢皮"
    },
    "55": {
        "pinyin": "heshouwu",
        "name": "何首乌"
    },
    "56": {
        "pinyin": "honghua",
        "name": "红花"
    },
    "57": {
        "pinyin": "hongkou",
        "name": "红蔻"
    },
    "58": {
        "pinyin": "houpu",
        "name": "厚朴"
    },
    "59": {
        "pinyin": "huaihua",
        "name": "槐花"
    },
    "60": {
        "pinyin": "huangbai",
        "name": "黄柏"
    },
    "61": {
        "pinyin": "huangjing",
        "name": "黄精"
    },
    "62": {
        "pinyin": "huangqin",
        "name": "黄芩"
    },
    "63": {
        "pinyin": "huomaren",
        "name": "火麻仁"
    },
    "64": {
        "pinyin": "huzhang",
        "name": "虎杖"
    },
    "65": {
        "pinyin": "jiangcan",
        "name": "僵蚕"
    },
    "66": {
        "pinyin": "jianghuang",
        "name": "姜黄"
    },
    "67": {
        "pinyin": "jineijin",
        "name": "鸡内金"
    },
    "68": {
        "pinyin": "jingjie",
        "name": "荆芥"
    },
    "69": {
        "pinyin": "jinqiancao",
        "name": "金钱草"
    },
    "70": {
        "pinyin": "jinyinhua",
        "name": "金银花"
    },
    "71": {
        "pinyin": "jixueteng",
        "name": "鸡血藤"
    },
    "72": {
        "pinyin": "juemingzi",
        "name": "决明子"
    },
    "73": {
        "pinyin": "kushen",
        "name": "苦参"
    },
    "74": {
        "pinyin": "laifuzi",
        "name": "莱菔子"
    },
    "75": {
        "pinyin": "lianqiao",
        "name": "连翘"
    },
    "76": {
        "pinyin": "lianzixin",
        "name": "莲子心"
    },
    "77": {
        "pinyin": "lingzhi",
        "name": "灵芝"
    },
    "78": {
        "pinyin": "lizhihe",
        "name": "荔枝核"
    },
    "79": {
        "pinyin": "longgu",
        "name": "龙骨"
    },
    "80": {
        "pinyin": "lulutong",
        "name": "路路通"
    },
    "81": {
        "pinyin": "luohanguo",
        "name": "罗汉果"
    },
    "82": {
        "pinyin": "luoshiteng",
        "name": "络石藤"
    },
    "83": {
        "pinyin": "maidong",
        "name": "麦冬"
    },
    "84": {
        "pinyin": "maiya",
        "name": "麦芽"
    },
    "85": {
        "pinyin": "mohanlian",
        "name": "墨旱莲"
    },
    "86": {
        "pinyin": "mudanpi",
        "name": "牡丹皮"
    },
    "87": {
        "pinyin": "muli",
        "name": "牡蛎"
    },
    "88": {
        "pinyin": "muxiang",
        "name": "木香"
    },
    "89": {
        "pinyin": "niuxi",
        "name": "牛膝"
    },
    "90": {
        "pinyin": "nvzhenzi",
        "name": "女贞子"
    },
    "91": {
        "pinyin": "paojiang",
        "name": "炮姜"
    },
    "92": {
        "pinyin": "peilan",
        "name": "佩兰"
    },
    "93": {
        "pinyin": "pugongying",
        "name": "蒲公英"
    },
    "94": {
        "pinyin": "puhuang",
        "name": "蒲黄"
    },
    "95": {
        "pinyin": "qianghuo",
        "name": "羌活"
    },
    "96": {
        "pinyin": "qianhu",
        "name": "前胡"
    },
    "97": {
        "pinyin": "qinghao",
        "name": "青蒿"
    },
    "98": {
        "pinyin": "quanxie",
        "name": "全蝎"
    },
    "99": {
        "pinyin": "renshen",
        "name": "人参"
    },
    "100": {
        "pinyin": "renshenqiepian",
        "name": "人参切片"
    },
    "101": {
        "pinyin": "roucongronggen",
        "name": "肉苁蓉根"
    },
    "102": {
        "pinyin": "roucongrongpian",
        "name": "肉苁蓉片"
    },
    "103": {
        "pinyin": "roudoukou",
        "name": "肉豆蔻"
    },
    "104": {
        "pinyin": "rougui",
        "name": "肉桂"
    },
    "105": {
        "pinyin": "sangpiaoxiao",
        "name": "桑螵蛸"
    },
    "106": {
        "pinyin": "sangshen",
        "name": "桑椹"
    },
    "107": {
        "pinyin": "sanqi",
        "name": "三七"
    },
    "108": {
        "pinyin": "shanyao",
        "name": "山药"
    },
    "109": {
        "pinyin": "shanzha",
        "name": "山楂"
    },
    "110": {
        "pinyin": "shanzhuyu",
        "name": "山茱萸"
    },
    "111": {
        "pinyin": "sharen",
        "name": "砂仁"
    },
    "112": {
        "pinyin": "shechuangzi",
        "name": "蛇床子"
    },
    "113": {
        "pinyin": "shegan",
        "name": "射干"
    },
    "114": {
        "pinyin": "shengma",
        "name": "升麻"
    },
    "115": {
        "pinyin": "shenqu",
        "name": "神曲"
    },
    "116": {
        "pinyin": "shichangpu",
        "name": "石菖蒲"
    },
    "117": {
        "pinyin": "shigao",
        "name": "石膏"
    },
    "118": {
        "pinyin": "shihu",
        "name": "石斛"
    },
    "119": {
        "pinyin": "shouwutengkuai",
        "name": "首乌藤块"
    },
    "120": {
        "pinyin": "shouwutengpian",
        "name": "首乌藤片"
    },
    "121": {
        "pinyin": "shuihonghuazi",
        "name": "水红花子"
    },
    "122": {
        "pinyin": "shuiniujiao",
        "name": "水牛角"
    },
    "123": {
        "pinyin": "suanzaoren",
        "name": "酸枣仁"
    },
    "124": {
        "pinyin": "taoren",
        "name": "桃仁"
    },
    "125": {
        "pinyin": "tiandong",
        "name": "天冬"
    },
    "126": {
        "pinyin": "tiankuizi",
        "name": "天葵子"
    },
    "127": {
        "pinyin": "tianmakuai",
        "name": "天麻块"
    },
    "128": {
        "pinyin": "tianmapian",
        "name": "天麻片"
    },
    "129": {
        "pinyin": "tiannanxing",
        "name": "天南星"
    },
    "130": {
        "pinyin": "tongcao",
        "name": "通草"
    },
    "131": {
        "pinyin": "tubiechong",
        "name": "土鳖虫"
    },
    "132": {
        "pinyin": "tusizi",
        "name": "菟丝子"
    },
    "133": {
        "pinyin": "wujiapi",
        "name": "五加皮"
    },
    "134": {
        "pinyin": "wulingzhi",
        "name": "五灵脂"
    },
    "135": {
        "pinyin": "wumei",
        "name": "乌梅"
    },
    "136": {
        "pinyin": "wuweizi",
        "name": "五味子"
    },
    "137": {
        "pinyin": "xiakucao",
        "name": "夏枯草"
    },
    "138": {
        "pinyin": "xiangfu",
        "name": "香附"
    },
    "139": {
        "pinyin": "xianhecao",
        "name": "仙鹤草"
    },
    "140": {
        "pinyin": "xiaohuixiang",
        "name": "小茴香"
    },
    "141": {
        "pinyin": "xinyi",
        "name": "辛夷"
    },
    "142": {
        "pinyin": "xixin",
        "name": "细辛"
    },
    "143": {
        "pinyin": "xuduan",
        "name": "续断"
    },
    "144": {
        "pinyin": "yejuhua",
        "name": "野菊花"
    },
    "145": {
        "pinyin": "yimucao",
        "name": "益母草"
    },
    "146": {
        "pinyin": "yinchen",
        "name": "茵陈"
    },
    "147": {
        "pinyin": "yiyiren",
        "name": "薏苡仁"
    },
    "148": {
        "pinyin": "yuanzhi",
        "name": "远志"
    },
    "149": {
        "pinyin": "yujin",
        "name": "郁金"
    },
    "150": {
        "pinyin": "yuzhupian",
        "name": "玉竹片"
    },
    "151": {
        "pinyin": "yuzhutiao",
        "name": "玉竹条"
    },
    "152": {
        "pinyin": "zelan",
        "name": "泽兰"
    },
    "153": {
        "pinyin": "zhebeimu",
        "name": "浙贝母"
    },
    "154": {
        "pinyin": "zhenzhumu",
        "name": "珍珠母"
    },
    "155": {
        "pinyin": "zhimu",
        "name": "知母"
    },
    "156": {
        "pinyin": "zhiqiaopian",
        "name": "枳壳片"
    },
    "157": {
        "pinyin": "zhiqiaotiao",
        "name": "枳壳条"
    },
    "158": {
        "pinyin": "zhishi",
        "name": "枳实"
    },
    "159": {
        "pinyin": "zhuru",
        "name": "竹茹"
    },
    "160": {
        "pinyin": "zicao",
        "name": "紫草"
    },
    "161": {
        "pinyin": "zihuadiding",
        "name": "紫花地丁"
    },
    "162": {
        "pinyin": "ziyuan",
        "name": "紫菀"
    }
}

In [13]:
def get_label_data() -> dict:
    return LABEL_DICT

In [14]:
def crop_image(img: np.ndarray, target_size: int, center: bool) -> np.ndarray:
    """
    这个函数用于对输入的图像进行裁剪操作。根据指定的target_size和center参数，
    函数会在图像中随机选择一个区域进行裁剪或者在图像中心位置进行裁剪，最终返回裁剪后的图像。
    """
    height, width = img.shape[:2]
    size = target_size
    if center:
        w_start = (width - size) / 2
        h_start = (height - size) / 2
    else:
        w_start = np.random.randint(0, width - size + 1)
        h_start = np.random.randint(0, height - size + 1)
    w_end = w_start + size
    h_end = h_start + size
    img = img[int(h_start):int(h_end), int(w_start):int(w_end), :]
    return img

In [15]:
def resize_short(img: np.ndarray, target_size: int) -> np.ndarray:
    """
    这个函数用于将输入的图像按照较短的边进行等比例缩放，使得较短的边的长度等于指定的target_size。
    函数首先计算缩放比例，然后根据计算出的比例对图像进行缩放操作，最终返回缩放后的图像。
    Args:
        img:
        target_size:

    Returns:

    """
    percent = float(target_size) / min(img.shape[0], img.shape[1])
    resized_width = int(round(img.shape[1] * percent))
    resized_height = int(round(img.shape[0] * percent))
    resized = cv2.resize(img, (resized_width, resized_height))
    return resized

In [16]:
image_file = r'Z:\BaiduNetdiskDownload\medicinalherb\train\imgs\aiye\aiye_0001.jpg'

In [17]:
import cv2

In [18]:
with open(image_file, 'rb') as file:
    image_bytes = file.read()

In [19]:
img = preprocess(image_bytes)
predictor = get_predictor()
result = run(predictor, [img])
res = result_process(result)
res

{'艾叶': 81.02, '茵陈': 5.71, '紫花地丁': 0.46, '白头翁': 0.34, '百部': 0.3}